In [42]:
import numpy as np
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

positive_reviews = [
    "Прекрасна атмосфера і вишукана кухня, завдяки яким вечір пройшов чудово."]

negative_reviews = [
    "Обслуговування залишає бажати кращого, а атмосфера застуджує до кісток."]

reviews = positive_reviews + negative_reviews
count_positive = len(positive_reviews)
count_negative = len(negative_reviews)
total_reviews = count_positive + count_negative

max_words_count = 500
max_text_len = 10
tokenizer = Tokenizer(num_words=max_words_count, filters='!–"—#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(reviews)

dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(reviews[0][:100])

data = tokenizer.texts_to_sequences(reviews)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(list(tokenizer.word_index.items()))

X = data_pad
Y = np.array([1] * count_positive + [0] * count_negative)

indices = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indices]
Y = Y[indices]

model = Sequential()
model.add(Embedding(max_words_count, 128, input_length=max_text_len))
model.add(GRU(128, return_sequences=True))
model.add(GRU(64, return_sequences=True))
model.add(GRU(32))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
history = model.fit(X, Y, batch_size=32, epochs=30)

[('прекрасна', 1), ('атмосфера', 2), ('і', 1), ('вишукана', 1), ('кухня', 1), ('завдяки', 1), ('яким', 1), ('вечір', 1), ('пройшов', 1), ('чудово', 1)]
Прекрасна атмосфера і вишукана кухня, завдяки яким вечір пройшов чудово.
[('атмосфера', 1), ('прекрасна', 2), ('і', 3), ('вишукана', 4), ('кухня', 5), ('завдяки', 6), ('яким', 7), ('вечір', 8), ('пройшов', 9), ('чудово', 10), ('обслуговування', 11), ('залишає', 12), ('бажати', 13), ('кращого', 14), ('а', 15), ('застуджує', 16), ('до', 17), ('кісток', 18)]
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 10, 128)           64000     
                                                                 
 gru_15 (GRU)                (None, 10, 128)           99072     
                                                                 
 gru_16 (GRU)                (None, 10, 64)            37248     
      

In [43]:
test_review = "Смакота та аромати з ресторану навівають гастрономічні відчуття."
test_data = tokenizer.texts_to_sequences([test_review])
test_data_pad = pad_sequences(test_data, maxlen=max_text_len)

prediction = model.predict(test_data_pad)
if prediction > 0.5:
    print("Результат: Позитивна рецензія")
else:
    print("Результат: Негативна рецензія")

1/1 [==============================] - 1s 1s/step
Результат: Позитивна рецензія


In [44]:
test_review_1 = "Обслуговування в ресторані залишає бажати кращого, а страви мають неприємний смак і вигляд.".lower()
test_data_1 = tokenizer.texts_to_sequences([test_review_1])
test_data_pad_1 = pad_sequences(test_data_1, maxlen=max_text_len)

prediction = model.predict(test_data_pad_1)
if prediction > 0.5:
    print("Результат: Позитивна рецензія")
else:
    print("Результат: Негативна рецензія")

1/1 [==============================] - 0s 25ms/step
Результат: Негативна рецензія
